In [2]:
import numpy as np
Pt = np.loadtxt("Pt/DOS.dat")
Fe = np.loadtxt("Fe/DOS.dat")

In [3]:
i = 0 
first = True
for row in Fe:
    E = row[0]
    if E > -0.5 and E < 2:
        if first:
            arr = row.copy().reshape(1,4)
            print(arr)
            first = False
        else:
            arr = np.append(arr,row.reshape((1,4)), axis = 0)


print(arr.shape)
data = arr

[[-0.412414  1.661276  2.045079  3.706355]]
(24, 4)


In [11]:
def interpolate(E, data):
    for row1 in data:
        if row1[0] > E:
            offset = E - row0[0]
            delta_E = row1[0]-row0[0]
            delta_spin_up = row1[1] - row0[1]
            delta_spin_down = row1[2] - row0[2]
            up_slope = delta_spin_up/delta_E
            down_slope = delta_spin_down/delta_E
            new_up = row0[1] + offset * up_slope
            new_down = row0[2] + offset * down_slope
            new_tot = new_up + new_down
            new_row = np.array([E, new_up, new_down, new_tot])
            return new_row
        row0 = row1

def trapz(f, x):
    vals = []
    for a,b,x0,x1 in zip(f[:-1], f[1:], x[:-1], x[1:]):
        dx = x1 - x0
        val = (a+b)*dx/2
        vals.append(val)
    vals_arr = np.array(vals)
    return vals_arr        

def integrate_DOS(E_DOS_intervals, data):
#     DOS_pdf = np.zeros((0, 0))
    E_0 = interpolate(E_DOS_intervals[0], data)
#     DOS_pdf = np.append(DOS_pdf, E_0.reshape((1,4)), axis = 0)
    DOS_pdf = E_0.reshape(1,4)
    for E in E_DOS_intervals[1:]:
        row = interpolate(E, data)
        DOS_pdf = np.append(DOS_pdf, row.reshape((1,4)), axis = 0)
    print(DOS_pdf)
    


    

In [12]:
step = 0.125
start = 0
end = 1.5

DOS_eV = np.arange(start, end + step, step)
print(DOS_eV)

integrate_DOS(DOS_eV, data)

[0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.    1.125 1.25  1.375
 1.5  ]
[[0.         1.40693935 0.66111287 2.06805222]
 [0.125      1.11906595 0.51668351 1.63574946]
 [0.25       0.75746733 0.46940016 1.22686748]
 [0.375      0.48377855 0.47774084 0.96151939]
 [0.5        0.3544825  0.53445576 0.88893826]
 [0.625      0.31765265 0.65771821 0.97537087]
 [0.75       0.30953607 0.88319388 1.19272995]
 [0.875      0.30449066 1.19593077 1.50042143]
 [1.         0.30013252 1.5362871  1.83641963]
 [1.125      0.29636175 1.87722015 2.1735819 ]
 [1.25       0.29163069 2.27008569 2.56171638]
 [1.375      0.28747552 2.67061899 2.95809451]
 [1.5        0.28382934 2.90497174 3.18880108]]
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [ ]:
step = 0.125
start = 0
end = 1.5

DOS_eV = np.arange(start - step/2, end + step*3/2, step)
print(DOS_eV)

# def one_trap(a,b,h):
#     area = (a+b)*h/2

def interpolate(E, data):
    for row1 in data:
        if row1[0] > E:
            offset = E - row0[0]
            delta_E = row1[0]-row0[0]
            delta_spin_up = row1[1] - row0[1]
            delta_spin_down = row1[2] - row0[2]
            up_slope = delta_spin_up/delta_E
            down_slope = delta_spin_down/delta_E
            new_up = row0[1] + offset * up_slope
            new_down = row0[2] + offset * down_slope
            new_tot = new_up + new_down
            new_row = np.array([E, new_up, new_down, new_tot])
            return new_row
        row0 = row1

In [ ]:



for E in DOS_eV:
    row = interpolate(E, arr)
    print(row)

In [ ]:
print(arr)